In [1]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df = pd.read_csv(os.path.join('WWR per cluster','WWR per cluster.gz'))

In [3]:
list(df)

['X',
 'Y',
 'CI',
 'GroundwaterDepth',
 'IrrigatedArea',
 'Population',
 'Region',
 'TDS',
 'geometry',
 'prec_1',
 'prec_2',
 'prec_3',
 'prec_4',
 'prec_5',
 'prec_6',
 'prec_7',
 'prec_8',
 'prec_9',
 'prec_10',
 'prec_11',
 'prec_12',
 'wind_1',
 'wind_2',
 'wind_3',
 'wind_4',
 'wind_5',
 'wind_6',
 'wind_7',
 'wind_8',
 'wind_9',
 'wind_10',
 'wind_11',
 'wind_12',
 'tmin_1',
 'tmin_2',
 'tmin_3',
 'tmin_4',
 'tmin_5',
 'tmin_6',
 'tmin_7',
 'tmin_8',
 'tmin_9',
 'tmin_10',
 'tmin_11',
 'tmin_12',
 'tmax_1',
 'tmax_2',
 'tmax_3',
 'tmax_4',
 'tmax_5',
 'tmax_6',
 'tmax_7',
 'tmax_8',
 'tmax_9',
 'tmax_10',
 'tmax_11',
 'tmax_12',
 'tavg_1',
 'tavg_2',
 'tavg_3',
 'tavg_4',
 'tavg_5',
 'tavg_6',
 'tavg_7',
 'tavg_8',
 'tavg_9',
 'tavg_10',
 'tavg_11',
 'tavg_12',
 'elevation',
 'lat',
 'long',
 'srad_1',
 'srad_2',
 'srad_3',
 'srad_4',
 'srad_5',
 'srad_6',
 'srad_7',
 'srad_8',
 'srad_9',
 'srad_10',
 'srad_11',
 'srad_12',
 'crop_share',
 'ETo_1',
 'ETo_2',
 'ETo_3',
 'ETo_4',

In [4]:
dff_water = df.groupby('Cluster').agg({'IrrigationWater': 'sum', 'PopulationWater': 'sum', 'TDS': 'mean', 'GroundwaterDepth': 'mean'})

In [5]:
dff_water.reset_index(inplace=True)

In [22]:
fig = go.Figure()
dff = dff_water

fig.add_trace(go.Bar(y=dff['Cluster'], x=dff['IrrigationWater']/1000000, orientation='h', name='Irrigation water extractions', marker=dict(color='#e66101', line=dict(width=0, color='Grey'))))
fig.add_trace(go.Bar(y=dff['Cluster'], x=dff['PopulationWater']/1000000, orientation='h', name='Population water extractions', marker=dict(color='#fdb863', line=dict(width=0, color='Grey'))))

fig.update_layout(barmode='stack', height=700, template='plotly_white',
                  yaxis={'categoryorder':'sum ascending', 'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                  xaxis={'title': 'Million cubic meters of water per year (Mm<sup>3</sup>/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="left",
                            x=0
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
fig.write_image('BaselineWater2.pdf')

In [26]:
dff_energy = df.groupby('Cluster').agg({'IrrigationPumpingEnergy': 'sum', 'IrrigationDesalinationEnergy': 'sum'})

In [27]:
dff_energy.reset_index(inplace=True)

In [34]:
fig = go.Figure()
dff = dff_energy

fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['IrrigationPumpingEnergy']/1000000, name='Pumping Energy', marker=dict(color='#fc8d62', line=dict(width=1, color='Grey'))))
fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['IrrigationDesalinationEnergy']/1000000, name='Desalination Energy', marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))

fig.update_layout(barmode='stack', width=900, template='plotly_white',
                  xaxis={'categoryorder':'sum descending', 'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                  yaxis={'title': 'Energy (GWh/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
fig.write_image('BaselineEnergy.pdf')

In [35]:
scenarios = np.array([x[1] for x in os.walk('.')][0])

In [36]:
scenarios = np.delete(scenarios, 0)

In [37]:
for scenario in scenarios:
    df = pd.read_csv(os.path.join(scenario, scenario.replace('agri','ag') + '.gz'))
    dff = df.groupby('Cluster').agg({'FinalPumpingEnergy': 'sum', 'FinalDesalinationEnergy': 'sum', 'FinalTreatmentEnergy': 'first'})
    dff.reset_index(inplace=True)
    dff['Cluster'] = dff['Cluster'].astype(int).astype(str)
    
    fig = go.Figure()

    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalTreatmentEnergy']/1000000, name='Treatment Energy', 
                         marker=dict(color='#8da0cb', line=dict(width=1, color='Grey'))))
    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalPumpingEnergy']/1000000, name='Pumping Energy', 
                         marker=dict(color='#fc8d62', line=dict(width=1, color='Grey'))))
    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalDesalinationEnergy']/1000000, name='Desalination Energy', 
                         marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))

    fig.update_layout(barmode='stack', width=900, template='plotly_white',
                      xaxis={'categoryorder':'sum descending', 'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                      yaxis={'title': 'Energy (GWh/yr)'},
                      legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                            ),
                     margin=dict(b=0, l=0, r=0, t=0))
    fig.write_image(os.path.join(scenario,'EnergyPerCluster2.pdf'))

In [34]:
dff['GroundwaterDepth']/dff['GroundwaterDepth'].min()

0     3.146789
1     2.879277
2     2.873699
3     1.151660
4     2.547099
5     3.077698
6     2.971967
7     1.059600
8     2.355352
9     2.622324
10    1.430181
11    2.708937
12    2.623005
13    1.439344
14    2.508038
15    1.000000
16    2.112251
17    2.953453
18    3.146789
19    2.622324
20    1.092606
21    3.119407
22    1.265100
23    3.146789
24    2.622324
25    2.622324
26    1.253972
27    3.047807
28    1.842023
29    1.048930
30    2.622324
31    1.845623
32    2.432178
33    2.958622
34    2.622324
35    1.093814
36    1.581023
37    2.172281
38    3.146789
39    2.574749
Name: GroundwaterDepth, dtype: float64

In [47]:
fig = go.Figure()
dff = dff_energy.merge(dff_water)


fig.add_trace(go.Scatter(x=dff[['IrrigationWater','PopulationWater']].sum(axis=1)/1000000, 
                         y=dff[['IrrigationPumpingEnergy','IrrigationDesalinationEnergy']].sum(axis=1)/1000000,
                         mode='markers',
                         text='Cluster: ' + dff['Cluster'].astype(int).astype(str),
                         marker=dict(size = dff['TDS']/dff['TDS'].min()*10,
                                    color=dff['GroundwaterDepth'],
                                    colorscale='viridis',
                                    showscale=True
                        )))
# fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['IrrigationDesalinationEnergy']/1000000, name='Desalination Energy', marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))

fig.update_layout(width=900, template='plotly_white',
                  xaxis={'categoryorder':'sum descending', 'title': 'Water (Mm<sup>3</sup>/yr)'}, 
                  yaxis={'title': 'Energy (GWh/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
# fig.write_image('BaselineEnergy.pdf')

In [48]:
dff.to_csv('DataForNexusPlot.csv')